<a id='toc'></a>
# Отчет по запланированным поездам

1. [Проверка планирования всех реальных поездов](#all_plan)
1. [Проверка совпадения четности номеров поездов и направления движения](#oddity)
2. [Анализ времен стоянок поездов на станциях смены локомотивов и бригад.](#stop_time)
   1. [Смена локомотивов](#stop_loco)
   2. [Смена бригад](#stop_team)
4. [Выявление случаев "близкой" отправки поездов с одной станции (с интервалом меньше 5 минут)](#dep_interval)
5. [Анализ отклонения запланированного на сутки количества поездов от данных АС ССП (данные средние по суткам).](#ssp)
6. [Проверка соответствия первого участка в запланированном маршруте и исходного факта](#info_plan_depart)
6. [Детальное сравнение количества поездов по Иркутску с данными ССП](#irk_ssp)
7. [Планирование сдвоенных поездов и поездов, составляющих сдвоенные](#Планирование-сдвоенных-поездов-и-поездов,-составляющих-сдвоенные)
6. [Создание отчета](#report)

### Функции для экспорта в HTML

In [265]:
report = ''
FOLDER = 'resources/'
REPORT_FOLDER = 'report/'
PRINT = True

In [266]:
def add_line(line, p=PRINT):    
    global report        
    if p:                
        if type(line) == pd.core.frame.DataFrame:
            print(line.to_string(index=False))
        elif type(line) == pd.core.series.Series:
            print(line.to_string())
        else:
            print(line)
    if type(line) == pd.core.frame.DataFrame:        
        report += ('%s<br>' % line.to_html(index=False))
    elif type(line) == pd.core.series.Series:
        report += ('%s<br>' % line.to_frame().reset_index().to_html(index=False))
    else:        
        report += ('%s<br>' % line)
    
def add_header(header, h=4, p=PRINT):
    global report
    report += ('<h%d>%s</h%d>' % (h, header, h))
    if p:
        print(header)

def add_image(filename, scale=0.4):
    global report
    report += ('<img src="%s" alt="%s" height="%d%%">' % (filename, filename, int(scale * 100)))

def create_report(filename):
    global report
    report = report.replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    html_string = '''
        <html>
            <head>
                <link rel="stylesheet" href="skeleton.css">
                <style>body{ margin:20 20; background:whitesmoke; }
                table {table-layout : fixed}
                </style>
            </head>
            <body>                
                %s
            </body>
        </html>''' % (report)
    f = open(filename,'w', encoding='utf-8-sig')
    f.write(html_string)
    f.close()
    print('Отчет сформирован: %s' % filename)

## Загрузка и подготовка данных

In [267]:
import numpy as np
import pandas as pd
import time, datetime
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rc('font', family='Tahoma')

pd.set_option('max_rows', 50)

time_format = '%d/%m %H:%M'

start_time = time.time()
current_time = pd.read_csv(FOLDER + 'current_time.csv').current_time[0]
twr          = pd.read_csv(FOLDER + 'team_work_region.csv', converters={'twr':str})
links        = pd.read_csv(FOLDER + 'link.csv', converters={'st_from':str, 'st_to':str})
stations     = pd.read_csv(FOLDER + 'station.csv', converters={'station':str})
train_info   = pd.read_csv(FOLDER + 'train_info.csv', converters={'train': str, 'st_from':str, 'st_to':str, 'joint':str,
                                                                 'start_st':str, 'end_st':str})
train_plan   = pd.read_csv(FOLDER + 'slot_train.csv', converters={'train': str, 'st_from':str, 'st_to':str})
loco_info    = pd.read_csv(FOLDER + 'loco_attributes.csv', converters={'train':str, 'loco':str, 'st_from':str, 'st_to':str})
loco_plan    = pd.read_csv(FOLDER + 'slot_loco.csv', converters={'train':str, 'loco':str, 'st_from':str, 'st_to':str})
team_info    = pd.read_csv(FOLDER + 'team_attributes.csv', converters={'team':str,'depot':str, 'oper_location':str, \
                                                     'st_from':str, 'st_to':str, 'loco':str, 'depot_st':str})
team_plan    = pd.read_csv(FOLDER + 'slot_team.csv', converters={'team':str,'loco':str, 'st_from':str, 'st_to':str})
loco_series  = pd.read_csv(FOLDER + 'loco_series.csv')
task         = pd.read_csv(FOLDER + 'task.csv', converters={'st_from':str, 'st_to':str, 'st_next':str})

st_names     = stations[['station', 'name', 'esr']].drop_duplicates().set_index('station')
team_info.regions = team_info.regions.apply(literal_eval)

print('Время составления отчета:', time.strftime(time_format, time.localtime()))
print('Время запуска планировщика: %s (%d)' % (time.strftime(time_format, time.localtime(current_time)), current_time))

Время составления отчета: 05/08 17:40
Время запуска планировщика: 03/08 12:20 (1470216024)


In [268]:
def nice_time(t):        
    return time.strftime(time_format, time.localtime(t)) if np.isnan(t) == False else None

def add_info(df):    
    if 'st_from' in df.columns:
        df['st_from_name'] = df.st_from.map(st_names.name)
    if 'st_to' in df.columns:
        df['st_to_name'] = df.st_to.map(st_names.name)
    if 'time_start' in df.columns:
        df['time_start_f'] = df.time_start.apply(lambda x: nice_time(x))
    if 'time_end' in df.columns:
        df['time_end_f'] = df.time_end.apply(lambda x: nice_time(x))
    if 'oper_location' in df.columns:
        df['oper_location_name'] = df.oper_location.map(st_names.name)    
        df.oper_location_name.fillna(0, inplace=True)
    if ('oper_location' in df.columns) & ('st_from' in df.columns) & ('st_to' in df.columns):        
        df['loc_name'] = df.oper_location_name
        df.loc[df.loc_name == 0, 'loc_name'] = df.st_from_name + ' - ' + df.st_to_name
    if 'oper_time' in df.columns:
        df['oper_time_f'] = df.oper_time.apply(lambda x: nice_time(x))    
    
# Добавляем во все таблицы названия станций на маршруте и времена отправления/прибытия в читабельном формате
add_info(train_plan), add_info(loco_plan), add_info(team_plan)
add_info(train_info), add_info(loco_info), add_info(team_info)

# Мержим таблицы _plan и _info для поездов, локомотивов и бригад
train_plan = train_plan.merge(train_info, on='train', suffixes=('', '_info'), how='left')
loco_plan = loco_plan.merge(loco_info, on='loco', suffixes=('', '_info'), how='left')
team_plan = team_plan.merge(team_info, on='team', suffixes=('', '_info'), how='left')

# Добавляем признаки поезда и бригады (реальный/локомотиво-резервный/объемный и реальная/фейковая)
train_plan['train_type'] = train_plan.train.apply(lambda x: str(x)[0])
team_plan['team_type'] = team_plan.team.apply(lambda x: 'Реальная' if str(x)[0] == '2' else 'Фейковая')

# Для локомотиво-резервных и объемных поездов заполняем номер
train_plan.loc[train_plan.train_type.isin(['8', '9']), 'number'] = train_plan.train.apply(lambda x: int(str(x)[-4:]))

# Добавляем подвязанные локомотив и бригаду в таблицы loco_plan и train_plan
if 'team' not in loco_plan.columns:
    loco_plan = pd.merge(loco_plan, team_plan.loc[team_plan.state == 1, ['team', 'loco', 'time_start']], 
                         on=['loco', 'time_start'], how='left')
if 'loco' not in train_plan.columns:
    train_plan = pd.merge(train_plan, loco_plan.loc[loco_plan.state.isin([0, 1]), ['loco', 'team', 'train', 'time_start']],
                     on=['train', 'time_start'], how='left')

<a id='all_plan'></a>

## Проверка планирования всех реальных поездов [ToC](#toc)

In [269]:
routes = pd.read_csv(FOLDER + 'routes.csv', dtype={'st_from':str, 'st_to':str, 'train':str})
add_info(routes)
routes.dropna(subset=['st_from_name', 'st_to_name'], how='any', inplace=True)
start_st = routes.drop_duplicates('train').set_index('train')
end_st = routes.drop_duplicates('train', keep='last').set_index('train')
train_info['first_st'] = train_info.train.map(start_st.st_from_name)
train_info['last_st'] = train_info.train.map(end_st.st_to_name)

In [270]:
train_info['in_plan'] = train_info.train.isin(train_plan.train)
a = train_info[(train_info.in_plan == False) 
               & (train_info.number > 1000) 
               & (train_info.number < 9000)
               & ((train_info.st_from != train_info.st_to) | (train_info.st_from == -1))\
              ].sort_values('number')
with pd.option_context('display.max_colwidth', 25):
    add_header('Всего %d реальных поездов (%.2f%%) не запланировано:' 
               % (a.train.count(), 100 * a.train.count() / train_info.train.count()))
    add_line(a[['train', 'number', 'ind434', 'loc_name', 'in_plan', 'first_st', 'last_st']])

Всего 41 реальных поездов (2.08%) не запланировано:
        train  number         ind434                  loc_name in_plan               first_st                   last_st
 230204399293    1214  8502-625-9142             ЛЕНА - ЯКУРИМ   False                   ЛЕНА                  МАРИИНСК
 200020659178    1400  2305-212-9142         ИЛАНСКАЯ - РЕШОТЫ   False               ИЛАНСКАЯ                  МАРИИНСК
 220207831073    1444  8515-836-9142      СЮЛЬБАН - НОВАЯ ЧАРА   False                СЮЛЬБАН                  МАРИИНСК
 200020814973    1562  8725-180-9678            ТАЙШЕТ - ТОРЕЯ   False                 ТАЙШЕТ                    ВАНИНО
 230203054378    2010  8630-971-9861   КИЖА - ПЕТРОВСКИЙ ЗАВОД   False                   КИЖА         НАХОДКА-ВОСТОЧНАЯ
 230204469682    2042  9700-899-8902             ШТУРМ - ТЫНДА   False                  ШТУРМ      КРАСНОЯРСК-ВОСТОЧНЫЙ
 200020644307    2044  9500-035-9628  ПРИАМУРСКАЯ - ПОСТ-ПО...   False            ПРИАМУРСКАЯ              В

<a id='oddity'></a>
## Проверка совпадения четности номеров поездов и направления движения [ToC](#toc)

In [271]:
add_header('Проверки по поездам', h=1, p=False)
add_header('Проверка совпадения четности номеров поездов и направления движения', h=2, p=False)

In [272]:
train_plan['dir'] = train_plan.link.map(links.set_index('link').dir)
train_plan['odevity'] = (((train_plan.number / 2).astype(int) * 2 == train_plan.number).astype(int) + 1) % 2
train_plan['check_odd'] = train_plan.dir == train_plan.odevity
cols = ['train', 'number', 'st_from_name', 'st_to_name', 'dir', 'odevity', 'check_odd']
fail_dir_number = train_plan.drop_duplicates(subset=['train', 'number']).loc[(train_plan.train_type.isin(['8', '9'])) &
                                                          (train_plan.check_odd == False), cols]
if fail_dir_number.empty == False:
    add_header('Четность номеров поездов и направления не совпадает для %d поездов (показаны первые 10):' %
         len(fail_dir_number.index))
    pd.set_option('display.max_colwidth', 35)
    add_line(fail_dir_number.head(10))
else:
    add_line('Все четности совпадают')

Все четности совпадают


<a id='stop_time'></a>
## Анализ времен стоянок поездов на станциях смены локомотивов и бригад [ToC](#toc)

#### Параметры для анализа

In [273]:
# Минимальное время стоянки поезда для смены локомотива
min_loco_stop = 1 * 3600 # 1 hour = 60 min

# Минимальное время стоянки поезда для смены бригады
min_team_stop = 15 * 60 # 15 min

# Горизонт проверки
hor = 24 * 3600

<a id='stop_loco'></a>
### Смена локомотивов [ToC](#toc)

In [274]:
add_header('Анализ смен локомотивов на маршрутах поездов', h=2, p=False)

#### Ищем станции смены локомотивов и считаем средние времена

In [275]:
train_plan.columns
train_plan.loco.fillna('-1', inplace=True)
train_plan.team.fillna('-1', inplace=True)

train_plan['train_end'] = train_plan.train != train_plan.train.shift(-1)
train_plan['loco_end'] = (train_plan.loco != train_plan.loco.shift(-1)) | (train_plan.train_end)
train_plan['team_end'] = (train_plan.team != train_plan.team.shift(-1)) | (train_plan.loco_end)

train_plan['stop_time'] = train_plan.time_start.shift(-1) - train_plan.time_end
train_plan['stop_time_h'] = np.round((train_plan.stop_time / 3600), 2)

train_plan['next_loco'] = train_plan.loco.shift(-1)
train_plan['next_team'] = train_plan.team.shift(-1)

cols = ['train', 'st_from_name', 'st_to_name', 'loco', 'team', 'stop_time_h', 'next_loco']
loco_change = train_plan[(train_plan.train_end == False) & (train_plan.loco_end == True)
                        & (train_plan.time_end < current_time + hor)]

In [276]:
add_header('Средние времена на смену локомотивов:')
add_line('- по всем сменам: %.2f ч.' % loco_change.stop_time_h.mean())
add_line('- по всем сменам с ненулевым временем: %.2f ч.' % loco_change[loco_change.stop_time_h > 0].stop_time_h.mean())

Средние времена на смену локомотивов:
- по всем сменам: 14.94 ч.
- по всем сменам с ненулевым временем: 15.01 ч.


#### Ищем поезда, у которых смена локомотивов происходит за нулевое время

In [277]:
cols = ['train', 'st_from_name', 'st_to_name', 'loco', 'next_loco', 'stop_time_h']
nill_stop_times = loco_change[loco_change.stop_time == 0]
if not nill_stop_times.empty:
    add_header('Всего %d поездов, для которых смена локомотивов происходит за нулевое время. Примеры:' 
               % nill_stop_times.train.count())
    add_line(nill_stop_times[cols].head())
    cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco', 'team']
    add_line('')
    add_header('Полный план по одному из таких поездов:')
    for train in nill_stop_times.train.values[:1]:
        add_line(train_plan[train_plan.train == train][cols])
else:
    add_header('Нет локомотивов, для которых смена бригад происходит за нулевое время')

Всего 2 поездов, для которых смена локомотивов происходит за нулевое время. Примеры:
        train         st_from_name           st_to_name          loco next_loco  stop_time_h
 200020706932          ИЗВЕСТКОВАЯ                 БИРА  200200104960        -1            0
 230204150916  КОМСОМОЛЬСК-НА-А...  КОМСОМОЛЬСК-НА-А...  200200028766        -1            0

Полный план по одному из таких поездов:
        train st_from_name   st_to_name time_start_f   time_end_f          loco          team
 200020706932       АРХАРА      ОБЛУЧЬЕ  03/08 01:45  03/08 04:01  200200104960  200200280543
 200020706932      ОБЛУЧЬЕ  ИЗВЕСТКОВАЯ  03/08 05:35  03/08 06:33  200200104960            -1
 200020706932  ИЗВЕСТКОВАЯ         БИРА  03/08 06:33  03/08 08:01  200200104960            -1
 200020706932         БИРА          NaN  03/08 08:01  03/08 08:55            -1            -1


#### Составляем статистику по всем станциям смены, загружаем список приоритетных станций смены

In [278]:
cols = ['train', 'st_from_name', 'st_to_name', 'loco', 'next_loco', 'stop_time_h']
no_nill_stops = loco_change[loco_change.stop_time > 0]
no_nill_stops[cols]
#no_nill_stops.groupby('st_to_name').stop_time_h.mean()
st_change = no_nill_stops.groupby('st_to_name').train.count().to_frame()\
            .join(no_nill_stops.groupby('st_to_name').stop_time_h.mean()).reset_index()
st_change['stop_time_h'] = st_change.stop_time_h.apply(lambda x: np.round(x, 2))
st_change.sort_values('train', ascending=False).head(10)
priority_change_stations = pd.read_csv(FOLDER + 'mandatory/priority_loco_change_stations.csv').st_name.values

#### Ищем поезда, у которых смена локомотивов происходит на неправильных станциях

In [279]:
# bad change stations
bad_changes = st_change[st_change.st_to_name.isin(priority_change_stations) == False].sort_values('train', ascending=False)
if not bad_changes.empty:
    add_header('Всего %d поездов, у которых смена локомотива, скорее всего, происходит на неправильных станциях' 
               % bad_changes.train.sum())
    add_header('Примеры таких станций:')
    add_line(bad_changes.head(10))
    add_line('')
    cols = ['train', 'st_from_name', 'st_to_name', 'loco', 'next_loco', 'stop_time_h']
    st = bad_changes.iloc[0].st_to_name
    add_line('Поезда, у которых смена локомотива происходит на станции %s:' % st)
    add_line(loco_change[loco_change.st_to_name == st][cols])

    train_plan['loco_info'] = train_plan.train.map(loco_info.drop_duplicates('train').set_index('train').loco)
    cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco', 'loc_name', 'loco_info']
    add_line('')
    add_header('Полный план по одному из таких поездов:')
    with pd.option_context('display.max_colwidth', 20):
        add_line(train_plan[train_plan.train == loco_change[loco_change.st_to_name == st].iloc[0].train][cols])
else:
    add_header('Нет поездов, у которых смена локомотивов происходит на неправильной станции')

Всего 149 поездов, у которых смена локомотива, скорее всего, происходит на неправильных станциях
Примеры таких станций:
   st_to_name  train  stop_time_h
         БИРА     62         2.42
 ВОЛОЧАЕВКА I     34         2.07
      ГОРЕЛЫЙ      8         6.30
  НОВЫЙ УРГАЛ      8         4.33
    ФЕВРАЛЬСК      4        15.37
  ДУБИНИНСКИЙ      3        15.49
      НАХОДКА      2         9.17
         ХАНИ      2         4.74
      СЕЛЬГОН      2        13.19
    ПОСТЫШЕВО      2         8.38

Поезда, у которых смена локомотива происходит на станции БИРА:
           train st_from_name st_to_name          loco next_loco  stop_time_h
    230202243548  ИЗВЕСТКОВАЯ       БИРА  200200089757        -1         1.89
    230200264580  ИЗВЕСТКОВАЯ       БИРА  200200088252        -1         1.69
    200020706932  ИЗВЕСТКОВАЯ       БИРА  200200104960        -1         0.00
    220209131375  ИЗВЕСТКОВАЯ       БИРА  200200067830        -1         2.03
    230202274091  ИЗВЕСТКОВАЯ       БИРА  2002000292

#### Ищем поезда со слишком долгой стоянкой для смены локомотивов

In [280]:
cols = ['train', 'st_from_name', 'st_to_name', 'loco', 'next_loco', 'stop_time_h']
long_change = loco_change[(loco_change.st_to_name.isin(priority_change_stations)) 
            & (loco_change.stop_time_h > 24)].sort_values('stop_time_h', ascending=False)
add_header('Всего %d случаев смены локомотива со стоянкой поезда более суток. Примеры:' % long_change.train.count())
add_line(long_change[cols].head(10))

Всего 78 случаев смены локомотива со стоянкой поезда более суток. Примеры:
        train st_from_name st_to_name          loco     next_loco  stop_time_h
 200020013134        КЛЮЧИ     РЕШОТЫ            -1  200200060245        95.47
 230204013383      ТАРСКАЯ  КАРЫМСКАЯ  200200081307  200200107156        95.03
 200020678345       ЧИТА I  КАРЫМСКАЯ  200200107173  200208841338        80.71
 230203464957       ЧИТА I  КАРЫМСКАЯ  200200101929  200200099330        80.10
 230204263453        ТАГУЛ     ТАЙШЕТ  200200101912  200200105038        77.66
 230204806501       ЧИТА I  КАРЫМСКАЯ  200200099696  200200045920        76.44
 230204361083      ТАРСКАЯ  КАРЫМСКАЯ  200200094033  200200095108        75.40
 200020890728   ЗАУДИНСКИЙ   УЛАН-УДЭ  200200072818  200200138580        74.87
 230204003363       ЧИТА I  КАРЫМСКАЯ  200200139935  200200098545        73.73
 230202901353       ЧИТА I  КАРЫМСКАЯ  200200107643  200200097957        71.63


<a id='stop_team'></a>
### Смена бригад [ToC](#toc)

In [281]:
add_header('Анализ смен бригад на маршрутах поездов', h=2, p=False)

In [282]:
team_change = train_plan[(train_plan.loco_end == False) & (train_plan.team_end == True)
                        & (train_plan.time_end < current_time + hor)]

In [283]:
add_header('Средние времена на смену бригады:')
add_line('- по всем сменам: %.2f ч.' % team_change.stop_time_h.mean())
add_line('- по всем сменам с ненулевым временем: %.2f ч.' % team_change[team_change.stop_time_h > 0].stop_time_h.mean())

Средние времена на смену бригады:
- по всем сменам: 4.82 ч.
- по всем сменам с ненулевым временем: 4.83 ч.


#### Ищем поезда, у которых смена бригады происходит за нулевое время

In [284]:
cols = ['train', 'st_from_name', 'st_to_name', 'team', 'next_team', 'stop_time_h']
nill_stop_times = team_change[team_change.stop_time == 0]
if not nill_stop_times.empty:
    add_header('Всего %d поездов, для которых смена бригад происходит за нулевое время. Примеры:' 
               % nill_stop_times.train.count())
    add_line(nill_stop_times[cols].head())
    cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco', 'team']
    add_line('')
    add_header('Полный план по одному из таких поездов:')
    for train in nill_stop_times.train.values[:1]:
        add_line(train_plan[train_plan.train == train][cols])
else:
    add_header('Нет поездов, для которых смена бригад происходит за нулевое время')

Всего 2 поездов, для которых смена бригад происходит за нулевое время. Примеры:
          train st_from_name   st_to_name          team     next_team  stop_time_h
 88880015574746         АМУР  ХАБАРОВСК I  777700003480  777700003469            0
 88880002064361     ИЛАНСКАЯ    ЗАОЗЕРНАЯ  777700003896  777700004777            0

Полный план по одному из таких поездов:
          train     st_from_name       st_to_name time_start_f   time_end_f          loco          team
 88880015574746    ВОЛОЧАЕВКА II         ДЕЖНЕВКА  04/08 00:20  04/08 00:35  200200097571  777700003480
 88880015574746         ДЕЖНЕВКА      ПРИАМУРСКАЯ  04/08 00:35  04/08 00:53  200200097571  777700003480
 88880015574746      ПРИАМУРСКАЯ  ПОСТ-ПОКРОВСКИЙ  04/08 00:53  04/08 00:57  200200097571  777700003480
 88880015574746  ПОСТ-ПОКРОВСКИЙ             АМУР  04/08 00:57  04/08 01:07  200200097571  777700003480
 88880015574746             АМУР      ХАБАРОВСК I  04/08 01:07  04/08 01:25  200200097571  777700003480
 88880

#### Ищем поезда со слишком долгой стоянкой для смены бригады

In [285]:
cols = ['train', 'st_from_name', 'st_to_name', 'team', 'next_team', 'stop_time_h']
long_change = team_change[team_change.stop_time_h > 6].sort_values('stop_time', ascending=False)[cols]
if not long_change.empty:
    add_header('Всего %d случаев смены бригад со стоянкой поезда более 6 часов. Примеры:' % long_change.train.count())
    add_line(long_change[cols].head(10))
    add_line('')
    cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco', 'team']
    add_header('Полный план по одному из таких поездов:')
    for train in long_change.train.values[:1]:
        add_line(train_plan[train_plan.train == train][cols])
    add_line('')
    add_header('Станции, на которых чаще всего происходили длительные стоянки на смену бригад:')
    add_line(long_change.st_to_name.value_counts().head(10))
else:
    add_header('Нет поездов, у которых смена бригады происходит более 6 часов')

Всего 714 случаев смены бригад со стоянкой поезда более 6 часов. Примеры:
           train st_from_name    st_to_name          team     next_team  stop_time_h
  88880015574865  НОВЫЙ УРГАЛ     ФЕВРАЛЬСК  200200271453  777700005169        21.69
 999912508012003     МОНГОХТО  ВЫСОКОГОРНАЯ  777700003749  777700004443        20.45
  88880013534553  НОВЫЙ УРГАЛ     ФЕВРАЛЬСК  200200191684  777700004856        16.25
 999912508012009     МОНГОХТО  ВЫСОКОГОРНАЯ  200200135960  200200222805        15.63
 999912508022011     МОНГОХТО  ВЫСОКОГОРНАЯ  777700004367  777700003929        15.45
 999912508022015     МОНГОХТО  ВЫСОКОГОРНАЯ  777700004512  777700003920        15.18
    230204450995       АМАЗАР        МОГОЧА  200200139828  777700003648        13.85
    220210372645        СУЛУК     ПОСТЫШЕВО  200200241908            -1        13.81
    230204555340       АМАЗАР        МОГОЧА  200200142701  777700003638        13.55
    230204508377       АМАЗАР        МОГОЧА  200200216927  777700003639     

<a id='dep_interval'></a>
## Поиск поездов с интервалом между отправлениями меньше допустимого [ToC](#toc)

In [286]:
add_header('Поиск поездов с интервалом между отправлениями меньше допустимого', h=2, p=False)

In [287]:
# Параметры

hor = 24 * 3600
min_time_delta = 5 * 60 # 5 minutes

In [288]:
# Функция, которая возвращает датафрейм с коллизиями

def check_time_collision(df):
    df['link_end'] = (df.st_from != df.st_from.shift(-1)) | (df.st_to != df.st_to.shift(-1))
    df['link_start'] = (df.st_from != df.st_from.shift(1)) | (df.st_to != df.st_to.shift(1))
    df.loc[df.link_end == False, 'time_to_next'] = df.time_start.shift(-1) - df.time_start
    df.loc[df.link_start == False, 'time_to_prev'] = df.time_start - df.time_start.shift(1)
    collisions = df.loc[(df.time_to_next < min_time_delta) | (df.time_to_prev < min_time_delta)]
    return collisions

In [289]:
add_line('Время начала планирования: %s' % nice_time(current_time))
cols = ['train', 'loco', 'team', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f']
train_plan['train_start'] = train_plan.train != train_plan.train.shift(1)
train_plan.loc[train_plan.stop_time != -1, 'loco_start'] = (train_plan.loco != train_plan.loco.shift(1)) | \
                                                            (train_plan.train_start)
train_plan.loc[train_plan.stop_time != -1, 'team_start'] = train_plan.team != train_plan.team.shift(1)
mask = ((train_plan.loco_start == True) | (train_plan.team_start == True)) &\
        (train_plan.time_start < current_time + hor) & (train_plan.time_start >= current_time)
assign_mask = (train_plan.loco != '-1') & (train_plan.team != '-1')
cols_to_sort = ['st_from_name', 'st_to_name', 'time_start']
start_times = train_plan.loc[mask].sort_values(cols_to_sort)
start_times_no_res = train_plan.loc[mask & (train_plan.train_type.isin(['2', '9']))].sort_values(cols_to_sort)
start_times_real = train_plan.loc[mask & (train_plan.train_type == '2')].sort_values(cols_to_sort)
start_times_assign = train_plan.loc[mask & assign_mask].sort_values(cols_to_sort)
start_times_assign_no_res = train_plan.loc[mask & assign_mask &
                                           (train_plan.train_type.isin(['2', '9']))].sort_values(cols_to_sort)
start_times_assign_real = train_plan.loc[mask & assign_mask & (train_plan.train_type == '2')].sort_values(cols_to_sort)

all_coll = check_time_collision(start_times)
no_res_coll = check_time_collision(start_times_no_res)
real_coll = check_time_collision(start_times_real)
all_assign_coll = check_time_collision(start_times_assign)
no_res_assign_coll = check_time_collision(start_times_assign_no_res)
real_assign_coll = check_time_collision(start_times_assign_real)
train_n = len(start_times.train.drop_duplicates().index)
res = pd.DataFrame([['Все поезда', 'Без учета резервных', 'Без учета резервных и фейковых'],                    
                    [len(all_coll.index), len(no_res_coll.index), len(real_coll.index)],                    
                    [len(all_assign_coll.index), len(no_res_assign_coll.index), len(real_assign_coll.index)]]).T
res.columns = ['Тип', 'Из всех поездов', 'Из всех с подвязкой']
add_header('Количество коллизий (интервал между поездами меньше %d минут):' % (min_time_delta / 60))
add_line(res)
add_header('\nРеальные и фейковые поезда с интервалами меньше %d минут (первые 20):' % (min_time_delta / 60))
add_line(no_res_coll[cols].head(20))

Время начала планирования: 03/08 12:20
Количество коллизий (интервал между поездами меньше 5 минут):
                 Тип Из всех поездов Из всех с подвязкой
          Все поезда             713                 617
 Без учета резервных               0                   0
 Без учета резерв...               0                   0

Реальные и фейковые поезда с интервалами меньше 5 минут (первые 20):
Empty DataFrame
Columns: [train, loco, team, st_from_name, st_to_name, time_start_f, time_end_f]
Index: []


<a id='ssp'></a>
## Сравнение количества запланированных поездов с данными АС ССП [ToC](#toc)

In [290]:
add_header('Сравнение количества запланированных поездов с данными АС ССП', h=2, p=False)

In [291]:
def count_volumes(full_plan, df_ssp):
    hor = 24 * 3600
    df_ssp.dep_dir.fillna(0, inplace=True)    
    df_ssp.loc[df_ssp.dep_dir == 0, 'depart'] = 0
    df_ssp.dropna(subset=['depart'], inplace=True)
    
    ssp_st_froms = df_ssp.loc[df_ssp.dep_dir == 0].station
    mask_time = (full_plan.time_start >= current_time) & (full_plan.time_start < current_time + hor)
    mask_type = full_plan.train_type.isin(['2', '9'])
    trains = full_plan.loc[mask_time & mask_type & full_plan.st_from_name.isin(ssp_st_froms)].\
                        groupby(['st_from_name', 'st_to_name']).train.count()
    df_ssp.loc[df_ssp.dep_dir == 0, 'st_from_name'] = df_ssp.station
    df_ssp = df_ssp.fillna(method='ffill')
    df_ssp['st_to_name'] = df_ssp.station
    replace_st_from_names = df_ssp.loc[df_ssp.dep_dir == 0, ['st_from_name', 'st_show_name']].drop_duplicates()
    df_ssp['st_from_show'] = df_ssp.st_from_name.map(replace_st_from_names.set_index('st_from_name').st_show_name)
    df_ssp['st_to_show'] = df_ssp.st_show_name    
    return trains.to_frame().join(df_ssp[['st_from_name', 'st_to_name', 'depart', 'st_from_show', 'st_to_show', 'dep_dir']].\
                                  set_index(['st_from_name', 'st_to_name'])).reset_index()

In [292]:
def show_barplot(df, road_name):
    df['delta'] = df.train - df.depart
    df['percent'] = np.round(100 * df.delta / df.depart, 2)
    df['st_from_short'] = df.st_from_show.apply(lambda x: str(x)[:25])
    df['st_to_short'] = df.st_to_show.apply(lambda x: str(x)[:25])
    df['link'] = df.st_from_short + ' - ' + df.st_to_short 
    
    print('%s железная дорога:' % road_name)
    print('Среднее и медиана абсолютного отклонения: %.2f, %.2f' % (df.delta.mean(), df.delta.median()))
    print('Среднее и медиана относительного отклонения (в процентах): %.2f%%, %.2f%%' % (df.percent.mean(), df.percent.median()))

    b = df.sort_values('delta', ascending=False)
    sns.set_style('whitegrid')
    sns.set_context('poster', font_scale=0.7, rc={'axes.labelsize': 18})
    c = sns.barplot(y='link', x='delta', data=df.sort_values('delta', ascending=False), palette='coolwarm')
    xlabel = '%s ж/д: отклонение от данных АС ССП по отправлению поездов на сутки' % road_name
    c.set(xlabel=xlabel, ylabel='')

In [293]:
VOL_PERCENT = 0.9

def show_two_barplots(df, road_name, save=False, btype='less'):
    df['st_from_short'] = df.st_from_show.apply(lambda x: str(x)[:25])
    df['st_to_short'] = df.st_to_show.apply(lambda x: str(x)[:25])
    df['link'] = df.st_from_short + ' - ' + df.st_to_short     
    sns.set_style('whitegrid')    
    sns.set_context('poster', font_scale=0.7, rc={'axes.titlesize':18, 'axes.labelsize':14})
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(14,20))    
    df.depart.fillna(0, inplace=True)    
    df['bottom'] = df.apply(lambda row: row.depart if row.train <= row.depart else row.train, axis=1)
    df['top'] = df.apply(lambda row: row.train if row.train <= row.depart else row.depart, axis=1)     
    
    sns.set_color_codes('pastel')
    sns.barplot(x='bottom', y='link', data=df[df.train <= VOL_PERCENT * df.depart].sort_values('bottom'), 
                label='Поезда из АС ССП', color="b", orient='h', ax=ax[0])
    sns.barplot(x='bottom', y='link', data=df[df.train * VOL_PERCENT > df.depart].sort_values('bottom'), 
                label='Поезда в результатах', color="r", orient='h', ax=ax[1])
    sns.set_color_codes('muted')
    sns.barplot(x='top', y='link', data=df[df.train <= VOL_PERCENT * df.depart].sort_values('bottom'), 
                label='Поезда в результатах', color="b", orient='h', ax=ax[0])
    sns.barplot(x='top', y='link', data=df[df.train * VOL_PERCENT > df.depart].sort_values('bottom'), 
                label='Поезда из АС ССП', color="r", orient='h', ax=ax[1])    
    ax[0].legend(ncol=1, loc="upper right", frameon=True)    
    ax[1].legend(ncol=1, loc="upper right", frameon=True)
    ax[0].set(xlabel='', title='Нехватка запланированных поездов')
    ax[1].set(xlabel='', title='Избыток запланированных поездов')
    
    sns.despine()
    if save:
        filename = road_name + '.png'
        fig.savefig(REPORT_FOLDER + filename, bbox_inches='tight')
        add_image(filename, scale=1.0)

In [294]:
def func(x):
    return np.round(np.sqrt(np.mean(x ** 2)), 2)

def print_ssp_stats(ssp, road_name):
    df = count_volumes(train_plan, ssp)
    df.rename(columns={'train':'planned', 'depart':'ssp'}, inplace=True)
    df.dropna(subset=['ssp'], inplace=True)
    df['delta'] = df.planned - df.ssp    
    cols = ['st_from_name', 'st_to_show', 'dep_dir', 'planned', 'ssp', 'delta']    
    add_header('Дорога %s' % road_name, h=3)
    add_header('Сравнение запланированного и "нормативного" количества поездов:')
    add_line(df.sort_values(['dep_dir', 'delta'])[cols])    
    add_header('\nСреднее отклонение по количеству поездов по направлениям:')
    add_line(df.groupby('dep_dir').delta.mean().apply(lambda x: np.round(x, 2)))
    add_header('\nСреднеквадратичное отклонение по направлениям:')
    add_line(df.groupby('dep_dir').delta.agg(func))

In [295]:
krs = pd.read_csv(FOLDER + 'mandatory/SSP_KRS.csv', sep=';')
vsib = pd.read_csv(FOLDER + 'mandatory/SSP_VSIB.csv', sep=';')
zab = pd.read_csv(FOLDER + 'mandatory/SSP_ZAB.csv', sep=';')
dvs = pd.read_csv(FOLDER + 'mandatory/SSP_DVS.csv', sep=';')

In [296]:
print_ssp_stats(krs, 'КРАС')

Дорога КРАС
Сравнение запланированного и "нормативного" количества поездов:
        st_from_name           st_to_show dep_dir  planned  ssp  delta
 КРАСНОЯРСК-ВОСТО...  КРАСНОЯРСК-СЕВЕРНЫЙ     неч       19   54    -35
            АЧИНСК I              БОГОТОЛ     неч       23   54    -31
             БОГОТОЛ             МАРИИНСК     неч       25   54    -29
            ИЛАНСКАЯ                  УЯР     неч       24   52    -28
              АБАКАН               ТАШЕБА     неч       13   26    -13
            САЯНСКАЯ          КОШУРНИКОВО     неч       25   24      1
            АЧИНСК I            АЧИНСК II     неч        6    4      2
            МАРИИНСК              БОГОТОЛ     чет       27   62    -35
             БОГОТОЛ             АЧИНСК I     чет       33   62    -29
 КРАСНОЯРСК-ВОСТО...                  УЯР     чет       39   66    -27
            АЧИНСК I       ЧЕРНОРЕЧЕНСКАЯ     чет       42   62    -20
            ИЛАНСКАЯ                 ЮРТЫ     чет       48   66    -18
 

In [297]:
print_ssp_stats(vsib, 'ВСИБ')

Дорога ВСИБ
Сравнение запланированного и "нормативного" количества поездов:
        st_from_name           st_to_show dep_dir  planned  ssp  delta
                ЗИМА          НИЖНЕУДИНСК     неч       28   70    -42
 ИРКУТСК-СОРТИРОВ...                 ЗИМА     неч       30   70    -40
              ТАЙШЕТ                 ЮРТЫ     неч       18   54    -36
         НИЖНЕУДИНСК               ТАЙШЕТ     неч       37   68    -31
          СЛЮДЯНКА I  ИРКУТСК-СОРТИРОВ...     неч       36   66    -30
           ВИХОРЕВКА                 ЧУНА     неч        9   18     -9
          НОВАЯ ЧАРА              ТАКСИМО     неч        1    8     -7
     СЕВЕРОБАЙКАЛЬСК              КИРЕНГА     неч        5   10     -5
              ТАЙШЕТ             САЯНСКАЯ     неч       17   22     -5
             ТАКСИМО           НОВЫЙ УОЯН     неч        5   10     -5
          НОВЫЙ УОЯН      СЕВЕРОБАЙКАЛЬСК     неч        6   10     -4
           ХРЕБТОВАЯ       ГИДРОСТРОИТЕЛЬ     неч       13   16     -3
 

In [298]:
print_ssp_stats(zab, 'ЗАБ')

Дорога ЗАБ
Сравнение запланированного и "нормативного" количества поездов:
        st_from_name           st_to_show dep_dir  planned  ssp  delta
         ШИМАНОВСКАЯ            МАГДАГАЧИ     неч       33   85    -52
           МАГДАГАЧИ          СКОВОРОДИНО     неч       45   84    -39
           БЕЛОГОРСК          ШИМАНОВСКАЯ     неч       45   83    -38
              ЧИТА I                ХИЛОК     неч       40   74    -34
              АРХАРА            БЕЛОГОРСК     неч       54   80    -26
             ГОРЕЛЫЙ                УРУША     неч       55   80    -25
      ШИЛКА-ТОВАРНАЯ            КАРЫМСКАЯ     неч       45   68    -23
             ТАРСКАЯ               ЧИТА I     неч       53   75    -22
               ХИЛОК     ПЕТРОВСКИЙ ЗАВОД     неч       51   70    -19
 ЧЕРНЫШЕВСК-ЗАБАЙ...       ШИЛКА-ТОВАРНАЯ     неч       52   70    -18
               УРУША               МОГОЧА     неч       61   70     -9
              МОГОЧА  ЧЕРНЫШЕВСК-ЗАБАЙ...     неч       62   70     -8
  

In [299]:
print_ssp_stats(dvs, 'ДВС')

Дорога ДВС
Сравнение запланированного и "нормативного" количества поездов:
        st_from_name           st_to_show dep_dir  planned  ssp  delta
              РУЖИНО             ГУБЕРОВО     неч       29   58    -29
           СИБИРЦЕВО       СПАССК-ДАЛЬНИЙ     неч       34   58    -24
           УССУРИЙСК            СИБИРЦЕВО     неч       34   58    -24
        ХАБАРОВСК II              ОБЛУЧЬЕ     неч       40   58    -18
              АРХАРА            БЕЛОГОРСК     неч       54   70    -16
         СМОЛЯНИНОВО              УГЛОВАЯ     неч       25   40    -15
           ГРОДЕКОВО       ГРОДЕКОВО(КИТ)     неч        2   13    -11
 КОМСОМОЛЬСК-НА-А...            ПОСТЫШЕВО     неч        3   10     -7
               ТЫНДА                 ХАНИ     неч        2    9     -7
              ВАНИНО         ВЫСОКОГОРНАЯ     неч       17   22     -5
            ГВОЗДЕВО            УССУРИЙСК     неч        1    4     -3
        ВЫСОКОГОРНАЯ    КОМСОМОЛЬСК-СОРТ.     неч       20   22     -2
  

In [300]:
# Пример построения barplot

#krs = pd.read_csv(FOLDER + 'mandatory/SSP_KRS.csv', sep=';')
#add_header('Красноярская дорога')
#try:
#    show_two_barplots(count_volumes(train_plan, krs), 'Красноярская', save=True, btype='less')
#except:
#    add_line('Красноярская дорога: ошибка в построении графика')

<a id='info_plan_depart'></a>
## Проверка соответствия первого участка в запланированном маршруте и исходного факта [ToC](#toc)

In [301]:
add_header('Проверка соответствия первого участка в запланированном маршруте и исходного факта', h=2, p=False)

In [302]:
cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'st_from_name_info', 'st_to_name_info', 'oper_time_f']
td_plan = train_plan[(train_plan.st_from_info.isnull() == False) 
                     & (train_plan.st_from_info != '-1')].drop_duplicates('train')
td_bad_track = td_plan[(td_plan.st_from != td_plan.st_from_info) | (td_plan.st_to != td_plan.st_to_info)]
add_header('Поезда, у которых первый участок в маршруте не совпадает с исходным (всего %d, показаны первые 10):' 
          % td_bad_track.train.count())
add_line(td_bad_track[cols])

td_bad_time = td_plan[(td_plan.time_start != td_plan.oper_time)]
add_header('\nПоезда, у которых время отправления на первый участок в маршруте не совпадает с фактическим (всего %d, показаны первые 10):' 
          % td_bad_time.train.count())
pd.set_option('display.max_colwidth', 19)
add_line(td_bad_time.sort_values('oper_time')[cols].head(10))

Поезда, у которых первый участок в маршруте не совпадает с исходным (всего 2, показаны первые 10):
        train st_from_name st_to_name time_start_f st_from_name_info st_to_name_info  oper_time_f
 230203328567       МОГОЧА     АМАЗАР  03/08 09:55            ЗИЛОВО     КСЕНЬЕВСКАЯ  03/08 03:06
 230202288495       МОГОЧА     АМАЗАР  03/08 09:55            ЗИЛОВО     КСЕНЬЕВСКАЯ  03/08 02:50

Поезда, у которых время отправления на первый участок в маршруте не совпадает с фактическим (всего 2, показаны первые 10):
        train st_from_name st_to_name time_start_f st_from_name_info st_to_name_info  oper_time_f
 230202288495       МОГОЧА     АМАЗАР  03/08 09:55            ЗИЛОВО     КСЕНЬЕВСКАЯ  03/08 02:50
 230203328567       МОГОЧА     АМАЗАР  03/08 09:55            ЗИЛОВО     КСЕНЬЕВСКАЯ  03/08 03:06


<a id='time_leaps'></a>
## Проверка скачков по времени назад [ToC](#toc)

In [303]:
add_header('Проверка скачков по времени назад', h=2, p=False)

In [304]:
train_plan['next_time_start'] = train_plan.time_start.shift(-1)
train_plan['next_time_start_f'] = train_plan.time_start_f.shift(-1)
train_plan['train_end'] = train_plan.train != train_plan.train.shift(-1)
cols = ['train', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'next_time_start_f']
leaps = train_plan[(train_plan.train_end == False) & (train_plan.next_time_start < train_plan.time_end)][cols]
if leaps.empty:
    add_header('Не найдено поездов со скачками по времени назад в плане')
else:
    add_header('Всего %d поездов со скачками по времени назад в плане. Примеры:' % leaps.train.count())
    add_line(leaps.head(10)[cols])

Не найдено поездов со скачками по времени назад в плане


<a id='irk_ssp'></a>

## Сравнение количества передаваемых в планировщик реальных поездов с ССП [ToC](#toc)

In [305]:
# Направления для проверки

test = [('ИРКУТСК-СОРТИРОВОЧНЫЙ', 'ГОНЧАРОВО'), ('ИРКУТСК-СОРТИРОВОЧНЫЙ', 'БАТАРЕЙНАЯ')]

In [306]:
add_header('Детальное сравнение количества поездов с данными ССП по станции %s' % test[0][0], h=2, p=False)

In [307]:
routes = pd.read_csv(FOLDER + 'routes.csv', dtype={'st_from':str, 'st_to':str, 'train':str})
add_info(routes)
routes['link_name'] = list(zip(routes.st_from_name, routes.st_to_name))
#def_tt = pd.read_csv(FOLDER + '/mandatory/travel_times_all_pairs.csv', sep=';').set_index(['st_from_name', 'st_to_name'])
def_tt = pd.read_csv(FOLDER + '/mandatory/travel_times.csv', index_col=0)

In [308]:
def get_arrive_time(row, station):
    if (row.oper == 'depart') | (row.oper_time >= current_time):
        start_time = row.oper_time
    else:
        start_time = current_time            
    return start_time + row.tt + row.lag

(test_st, test_st_dir) = test[0]
train_info['st_loc_name'] = train_info.oper_location.map(st_names.name)
train_info.st_loc_name.fillna(train_info.st_from_name, inplace=True)
#train_info['tt'] = train_info.st_loc_name.apply(lambda x: def_tt.ix[x, test_st].tt)
train_info['tt'] = train_info.st_loc_name.apply(lambda x: 48 * 3600 if type(x) == float else def_tt[x][test_st])

train_info['lag'] = np.round((train_info.tt / 6) * 1.5)
train_info['arr_time'] = train_info[['oper', 'oper_time', 'tt', 'lag']]\
                                    .apply(lambda row: get_arrive_time(row, test_st), axis=1)
train_info['arr_time_f'] = train_info.arr_time.apply(nice_time)
train_info['plan_time'] = train_info.train.map(train_plan[train_plan.st_from_name == test_st].set_index('train').time_start)
train_info['plan_time_f'] = train_info.plan_time.apply(nice_time)
train_info['delta'] = train_info.plan_time - train_info.arr_time
train_info['delta_h'] = np.round((train_info.delta / 3600), 2)

train_plan['link_name'] = list(zip(train_plan.st_from_name, train_plan.st_to_name))

dir_trains = train_info[(train_info.number >= 1000)
                        & (train_info.train.isin(routes[routes.link_name == (test_st, test_st_dir)].train))]
cols = ['train', 'number', 'oper', 'st_loc_name', 'tt', 'lag', 'oper_time_f', 'arr_time_f', 'plan_time_f', 'delta_h']
dir_trains = dir_trains[dir_trains.arr_time < current_time + 24 * 3600].sort_values('arr_time')
#a[cols]
add_header('Всего %d поездов, по которым ожидается проследование в направлении %s - %s в первые сутки планирования'
           % (dir_trains.train.count(), test_st, test_st_dir))

Всего 72 поездов, по которым ожидается проследование в направлении ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО в первые сутки планирования


In [309]:
no_plan = dir_trains[dir_trains.plan_time.isnull()][cols]
pd.set_option('display.max_colwidth', 40)
add_header('Всего %d поездов на направлении %s - %s, которых вообще нет в плане' % (no_plan.train.count(), test_st, test_st_dir))
add_line(no_plan)

Всего 7 поездов на направлении ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО, которых вообще нет в плане
        train  number    oper           st_loc_name     tt    lag  oper_time_f   arr_time_f plan_time_f  delta_h
 230204252850    9260   ready           НИЖНЕУДИНСК  31566   7892  03/08 10:08  03/08 23:18        None      NaN
 230204273751    9210   ready           НИЖНЕУДИНСК  31566   7892  03/08 10:08  03/08 23:18        None      NaN
 200020663666    9226   ready           НИЖНЕУДИНСК  31566   7892  03/08 00:20  03/08 23:18        None      NaN
 200020814973    1562  depart                ТАЙШЕТ  41741  10435  03/08 11:23  04/08 01:52        None      NaN
 230204433621    2808  depart  КРАСНОЯРСК-ВОСТОЧНЫЙ  66811  16703  03/08 10:52  04/08 10:03        None      NaN
 200020644489    2090  depart  КРАСНОЯРСК-ВОСТОЧНЫЙ  66811  16703  03/08 11:09  04/08 10:20        None      NaN
 200020678313    9202  depart        ЧЕРНОРЕЧЕНСКАЯ  78850  19712  03/08 08:44  04/08 12:06        None      NaN


In [310]:
plan_day = dir_trains[dir_trains.plan_time < current_time + 24 * 3600][cols]
add_header('Всего %d поездов (%.2f%%), по которым запланировано проследование в направлении %s - %s в первые сутки'
          % (plan_day.train.count(), 100 * plan_day.train.count() / dir_trains.train.count(), test_st, test_st_dir))

Всего 37 поездов (51.39%), по которым запланировано проследование в направлении ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО в первые сутки


In [311]:
# sns.set(context='notebook', style='whitegrid')
# sns.set_color_codes('dark')
# plt.figure(figsize=(10, 5))
# sns.kdeplot(dir_trains[dir_trains.train.isin(plan_day.train) == False].dropna(subset=['delta']).delta / 3600, shade=True)

delta_lim = 3 * 3600
late_trains = dir_trains[dir_trains.delta > delta_lim]
late_trains.sort_values('delta', ascending=False)[cols]
add_header('Всего %d поездов (%.2f%%), у которых запланированное время проследования на участке %s - %s сильно сдвинуто вперед' 
           % (late_trains.train.count(), 100 * late_trains.train.count() / dir_trains.train.count(), test_st, test_st_dir))
add_line(late_trains.sort_values('delta', ascending=False)[cols])

Всего 47 поездов (65.28%), у которых запланированное время проследования на участке ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО сильно сдвинуто вперед
        train  number    oper            st_loc_name     tt    lag  oper_time_f   arr_time_f  plan_time_f  delta_h
 230204263453    2712  depart              ЩЕТИНКИНО  70779  17695  03/08 08:53  04/08 09:27  07/08 16:03    78.60
 230204665234    1252  depart               САЯНСКАЯ  59325  14831  03/08 12:08  04/08 08:43  06/08 22:12    61.47
 200020659004    2714  depart              ЩЕТИНКИНО  70779  17695  03/08 09:08  04/08 09:42  06/08 21:27    59.75
 200020644129    2518  arrive   КРАСНОЯРСК-ВОСТОЧНЫЙ  66811  16703  03/08 09:57  04/08 11:32  06/08 21:37    58.09
 200020683571    1748   ready              СУХОВСКАЯ   1447    362  02/08 22:50  03/08 12:50  05/08 20:07    55.29
 230203081134    2676  depart               САЯНСКАЯ  59325  14831  03/08 11:14  04/08 07:49  06/08 02:02    42.21
 200020639408    2622  depart              ИРБЕЙСКАЯ  

In [312]:
plan_cols = ['train', 'oper', 'oper_time_f', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco', 'team']
add_header('Пример плана по одному из поездов, формированием НЕ на станции %s:' % test_st)
with pd.option_context('display.max_colwidth', 15):
    add_line(train_plan[train_plan.train == late_trains[late_trains.st_loc_name != test_st]\
                        .sort_values('delta', ascending=False).iloc[0].train][plan_cols])

Пример плана по одному из поездов, формированием НЕ на станции ИРКУТСК-СОРТИРОВОЧНЫЙ:
        train    oper  oper_time_f    st_from_name      st_to_name time_start_f   time_end_f          loco          team
 230204263453  depart  03/08 08:53       ЩЕТИНКИНО        САЯНСКАЯ  03/08 08:53  03/08 12:15  200200101912  200200132340
 230204263453  depart  03/08 08:53        САЯНСКАЯ       ИРБЕЙСКАЯ  03/08 14:55  03/08 15:55  200200101912  200200188637
 230204263453  depart  03/08 08:53       ИРБЕЙСКАЯ           ТАГУЛ  03/08 15:55  03/08 20:19  200200101912  200200188637
 230204263453  depart  03/08 08:53           ТАГУЛ          ТАЙШЕТ  03/08 20:19  03/08 21:02  200200101912  200200188637
 230204263453  depart  03/08 08:53          ТАЙШЕТ      БАЙРОНОВКА  07/08 02:42  07/08 03:03  200200105038            -1
 230204263453  depart  03/08 08:53      БАЙРОНОВКА         АЛЗАМАЙ  07/08 03:03  07/08 03:58  200200105038            -1
 230204263453  depart  03/08 08:53         АЛЗАМАЙ     НИЖНЕУДИНСК 

In [313]:
slot = pd.read_csv(FOLDER + 'slot.csv', dtype={'st_from':str, 'st_to':str})
add_info(slot)
slot['dt_start'] = slot.time_start.apply(datetime.datetime.fromtimestamp)

In [314]:
test = [('ТАЙШЕТ', 'ТОРЕЯ')]
(test_st, test_st_dir) = test[0]
plan_cols = ['train', 'st_from_name', 'st_to_name', 'dt_start', 'loco', 'team']
train_plan['dt_start'] = train_plan.time_start.apply(datetime.datetime.fromtimestamp)
a = train_plan[(train_plan.st_from_name == test_st) 
           & (train_plan.st_to_name == test_st_dir) & (train_plan.train_type.isin(['2', '9']))
           & (train_plan.time_start >= current_time) & (train_plan.time_end < current_time + 24 * 3600)]\
    .sort_values('dt_start')[plan_cols]
dep_volume = a.set_index('dt_start').resample('1H', how='count').train
dep_volume

dt_start
2016-08-03 15:00:00    2
2016-08-03 16:00:00    1
2016-08-03 17:00:00    0
2016-08-03 18:00:00    0
2016-08-03 19:00:00    0
2016-08-03 20:00:00    0
2016-08-03 21:00:00    0
2016-08-03 22:00:00    2
2016-08-03 23:00:00    1
2016-08-04 00:00:00    1
2016-08-04 01:00:00    0
2016-08-04 02:00:00    0
2016-08-04 03:00:00    0
2016-08-04 04:00:00    2
2016-08-04 05:00:00    2
2016-08-04 06:00:00    0
2016-08-04 07:00:00    1
Freq: H, Name: train, dtype: int64

In [315]:
slot_volume = slot[(slot.st_from_name == test_st) & (slot.st_to_name == test_st_dir)]\
    .set_index('dt_start').resample('1H', how='count').slot
dep_slot = dep_volume.to_frame().join(slot_volume)

In [316]:
# slot_volume = slot[(slot.st_from_name == test_st) & (slot.st_to_name == test_st_dir)]\
#     .set_index('dt_start').resample('300s')
# slot_volume.dropna(subset=['slot'], inplace=True)
# slot_volume['slot'] = slot_volume.slot.apply(int)
# slot_volume = slot_volume.reset_index().set_index('slot')
# slot_volume

In [317]:
a = dep_slot[dep_slot.train > dep_slot.slot]
print(a)

Empty DataFrame
Columns: [train, slot]
Index: []


In [318]:
pd.set_option('display.max_colwidth', 40)
cols = ['train', 'st_from_name', 'st_to_name', 'dt_start']
for dt in a.index:
    dt_next = dt + datetime.timedelta(0, 3600, 0)
    print(dt, dt_next)
    print(train_plan[(train_plan.train_type.isin(['2', '9']))
                    & (train_plan.st_from_name == test_st) & (train_plan.st_to_name == test_st_dir)
                    & (train_plan.dt_start >= dt) & (train_plan.dt_start < dt_next)].sort_values('dt_start')[cols])
    print(slot[(slot.st_from_name == test_st) & (slot.st_to_name == test_st_dir)
                    & (slot.dt_start >= dt) & (slot.dt_start < dt_next)].sort_values('dt_start')[['slot', 'dt_start']])
    print('-------')

[В начало](#toc)
## Планирование сдвоенных поездов и поездов, составляющих сдвоенные

0. Сдвоенные поезда - это поезда, которые указаны в атрибуте `joint` у каких-либо других поездов.
1. Сдвоенные поезда с точки зрения планирования ничем не отличаются от обычных поездов. Проверяется, что сдвоенные поезда планируются до своей конечной станции.
2. Составляющие поезда должны планироваться от конечной станции сдвоенного поезда и только после прибытия сдвоенного поезда на конечную станцию.

In [319]:
add_header('Планирование сдвоенных поездов и поездов, составляющих сдвоенные', h=2, p=False)

In [320]:
train_info['is_arrive'] = train_info.last_st == train_info.loc_name
train_plan['last_st_info'] = train_plan.train.map(train_info.set_index('train').last_st)
joints = [t for t in train_info.joint.unique() if t != '-1']
info_joints = train_info[train_info.train.isin(joints)]
joints_to_plan = info_joints[info_joints.is_arrive == False]
joints_planned = train_plan[train_plan.train.isin(joints_to_plan.train)]
add_header('Всего запланировано %d сдвоенных поездов из %d корректно переданных (%.2f%%). Примеры незапланированных поездов:' 
           % (joints_planned.train.count(), joints_to_plan.train.count(), 
              100 * joints_planned.drop_duplicates('train').train.count() / joints_to_plan.train.count()))
cols = ['train', 'number', 'ind434', 'oper', 'oper_time_f', 'loc_name']
pd.set_option('display.max_colwidth', 50)
add_line(joints_to_plan.head(10)[cols])

Всего запланировано 1 сдвоенных поездов из 31 корректно переданных (3.23%). Примеры незапланированных поездов:
        train  number         ind434    oper  oper_time_f                   loc_name
 200020738171    9218  9715-646-9826  depart  03/08 08:55          РУЖИНО - ШМАКОВКА
 200020744284    9209  9826-531-9700  depart  03/08 10:34  СПАССК-ДАЛЬНИЙ - ШМАКОВКА
 200020772007    9215  9525-472-9513  depart  03/08 08:01     ТАЛДАН - БОЛЬШОЙ НЕВЕР
 200020806055    9210  9212-203-9320  depart  03/08 11:29       НИЖНЕУДИНСК - УДА II
 200020668853    9209  9702-304-9614  depart  03/08 10:11     ЗАВИТАЯ - БЕЛОГОРСК II
 200020767156    9219  9628-080-9614  depart  03/08 07:21         БИРА - ИЗВЕСТКОВАЯ
 200020698338    9203  9826-529-9700  depart  03/08 11:31       ГУБЕРОВО - ЛУЧЕГОРСК
 200020726330    9220  9700-345-9828  arrive  03/08 12:13                     РУЖИНО
 200020714370    9208  9491-004-9509  depart  03/08 06:23       ЗИЛОВО - КСЕНЬЕВСКАЯ
 200020772011    9213  9826-532-9700  d

In [321]:
tl = joints_planned.drop_duplicates('train', keep='last')
tl_no_end = tl[tl.st_to_name != tl.last_st_info]
add_header('Всего %d сдвоенных поездов из %d запланированных (%.2f%%) запланированы НЕ до конца маршрута. Примеры:' 
           % (tl_no_end.train.count(), joints_planned.drop_duplicates('train').train.count(), 
              100 * tl_no_end.train.count() / joints_planned.drop_duplicates('train').train.count()))
cols = ['train', 'number', 'ind434', 'st_from_name', 'st_to_name', 'last_st_info']
add_line(tl_no_end.head(10)[cols])

Всего 0 сдвоенных поездов из 1 запланированных (0.00%) запланированы НЕ до конца маршрута. Примеры:
Empty DataFrame
Columns: [train, number, ind434, st_from_name, st_to_name, last_st_info]
Index: []


In [322]:
ind = '9114-084-8176'
pd.set_option('display.max_colwidth', 20)
loco_info['ser_name'] = loco_info.series.map(loco_series.set_index('ser_id').ser_name)
train_id = train_info[train_info.ind434 == ind].train.values[0]
cols = ['train', 'number', 'st_from_name', 'st_to_name', 'time_start_f', 'time_end_f', 'loco_n', 'team_n']
train_plan['loco_n'] = train_plan.loco.map(loco_info.set_index('loco').number)
train_plan['team_n'] = train_plan.team.map(team_info.set_index('team').number)
add_line(train_plan[train_plan.train == train_id][cols])
add_line(train_info[train_info.train == train_id][['train', 'loc_name', 'oper_time_f', 'oper']])
st1 = 'ТАКСИМО'
st2 = 'КУАНДА'
add_line(train_plan[(train_plan.st_from_name == st1) & (train_plan.st_to_name == st2)
          & (train_plan.time_start >= current_time)
          & (train_plan.time_start < current_time + 24 * 3600)].sort_values('time_start')[cols])

        train  number   st_from_name     st_to_name time_start_f   time_end_f  loco_n      team_n
 230202979382    2627   МЕЖДУРЕЧЕНСК       БИСКАМЖА  03/08 13:45  03/08 17:36    1018  8875007422
 230202979382    2627       БИСКАМЖА        НАНХЧУЛ  03/08 17:36  03/08 17:59    1018  8875007422
 230202979382    2627        НАНХЧУЛ         ХАБЗАС  03/08 17:59  03/08 18:22    1018  8875007422
 230202979382    2627         ХАБЗАС          АСКИЗ  03/08 18:22  03/08 19:57    1018  8875007422
 230202979382    2627          АСКИЗ        КАМЫШТА  03/08 19:57  03/08 21:17    1018  8875007422
 230202979382    2627        КАМЫШТА          ХОНЫХ  03/08 21:17  03/08 22:42    1018  8875007422
 230202979382    2627          ХОНЫХ   ОРОСИТЕЛЬНЫЙ  03/08 22:42  03/08 22:52    1018  8875007422
 230202979382    2627   ОРОСИТЕЛЬНЫЙ      ЕРБИНСКАЯ  04/08 01:55  04/08 04:29     NaN         NaN
 230202979382    2627      ЕРБИНСКАЯ       ТУМАННЫЙ  04/08 04:29  04/08 05:13     NaN         NaN
 230202979382    262

<a id='report'></a>
### Экспорт в HTML [ToC](#toc)

In [323]:
filename = REPORT_FOLDER + 'train_report_' + time.strftime('%Y%m%d_%H%M%S', time.localtime(time.time())) + '.html'
create_report(filename)

Отчет сформирован: report/train_report_20160805_174021.html
